In [19]:
#Importing libraries 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
print ('modules loaded')


modules loaded


In [99]:
#Loading the dataset
url = "C:\\Users\\ranas\\OneDrive\\Downloads\\archive (9)\\data"
p_url = Path(url)
for file in os.listdir(url):
    print(file,len(os.listdir(url+"\\"+file)))
    for i in os.listdir(url+"\\"+file):
        print(i,len(os.listdir(url+"\\"+file+"\\"+i)))

testing_data 36
0 28
1 28
2 28
3 28
4 28
5 28
6 28
7 28
8 28
9 28
A 28
B 28
C 28
D 28
E 28
F 28
G 28
H 28
I 28
J 28
K 28
L 28
M 28
N 28
O 28
P 28
Q 28
R 28
S 28
T 28
U 28
V 28
W 28
X 28
Y 28
Z 28
training_data 36
0 573
1 573
2 573
3 573
4 573
5 573
6 573
7 573
8 573
9 573
A 573
B 573
C 573
D 573
E 573
F 573
G 573
H 573
I 573
J 573
K 573
L 573
M 573
N 573
O 573
P 573
Q 573
R 573
S 573
T 573
U 573
V 573
W 573
X 573
Y 573
Z 573


In [78]:
data = tf.keras.utils.image_dataset_from_directory(
    url+"\\training_data",
    labels="inferred",
    image_size=(50, 32),
    batch_size=16,
    validation_split=0.2,  # Split the dataset into training and validation
    subset="training",      # Specify "training" or "validation"
    seed=42
)
print(data.class_names)

Found 20628 files belonging to 36 classes.
Using 16503 files for training.
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [82]:
#Loading the model 
base_model = tf.keras.applications.efficientnet.EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=(50,32,3),
    pooling='max',
    classes=1000,
    classifier_activation='softmax'
)
x=base_model.output
print(tf.shape(x))
x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.45, seed=123)(x)        
output=Dense(36, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(learning_rate=.001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 


KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 1536], name='tf.compat.v1.shape_8/Shape:0', description="created by layer 'tf.compat.v1.shape_8'")


In [83]:
model.build([None,50,32,3])
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_33 (InputLayer)       [(None, 50, 32, 3)]          0         []                            
                                                                                                  
 rescaling_64 (Rescaling)    (None, 50, 32, 3)            0         ['input_33[0][0]']            
                                                                                                  
 normalization_32 (Normaliz  (None, 50, 32, 3)            7         ['rescaling_64[0][0]']        
 ation)                                                                                           
                                                                                                  
 rescaling_65 (Rescaling)    (None, 50, 32, 3)            0         ['normalization_32[0][0

                                                                                                  
 block2a_expand_conv (Conv2  (None, 25, 16, 144)          3456      ['block1b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block2a_expand_bn (BatchNo  (None, 25, 16, 144)          576       ['block2a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block2a_expand_activation   (None, 25, 16, 144)          0         ['block2a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block2a_d

 block2c_expand_conv (Conv2  (None, 13, 8, 192)           6144      ['block2b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block2c_expand_bn (BatchNo  (None, 13, 8, 192)           768       ['block2c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block2c_expand_activation   (None, 13, 8, 192)           0         ['block2c_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block2c_dwconv (DepthwiseC  (None, 13, 8, 192)           1728      ['block2c_expand_activation[0]
 onv2D)   

 D)                                                                                               
                                                                                                  
 block3b_expand_bn (BatchNo  (None, 7, 4, 288)            1152      ['block3b_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block3b_expand_activation   (None, 7, 4, 288)            0         ['block3b_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block3b_dwconv (DepthwiseC  (None, 7, 4, 288)            7200      ['block3b_expand_activation[0]
 onv2D)                                                             [0]']                         
          

 block4a_expand_conv (Conv2  (None, 7, 4, 288)            13824     ['block3c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block4a_expand_bn (BatchNo  (None, 7, 4, 288)            1152      ['block4a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block4a_expand_activation   (None, 7, 4, 288)            0         ['block4a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block4a_dwconv_pad (ZeroPa  (None, 9, 5, 288)            0         ['block4a_expand_activation[0]
 dding2D) 

...

                                                                                                  
 block6a_expand_bn (BatchNo  (None, 4, 2, 816)            3264      ['block6a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6a_expand_activation   (None, 4, 2, 816)            0         ['block6a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block6a_dwconv_pad (ZeroPa  (None, 7, 5, 816)            0         ['block6a_expand_activation[0]
 dding2D)                                                           [0]']                         
                                                                                                  
 block6a_d

 block6c_expand_bn (BatchNo  (None, 2, 1, 1392)           5568      ['block6c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6c_expand_activation   (None, 2, 1, 1392)           0         ['block6c_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block6c_dwconv (DepthwiseC  (None, 2, 1, 1392)           34800     ['block6c_expand_activation[0]
 onv2D)                                                             [0]']                         
                                                                                                  
 block6c_bn (BatchNormaliza  (None, 2, 1, 1392)           5568      ['block6c_dwconv[0][0]']      
 tion)    

                                                                                                  
 block6e_expand_bn (BatchNo  (None, 2, 1, 1392)           5568      ['block6e_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6e_expand_activation   (None, 2, 1, 1392)           0         ['block6e_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block6e_dwconv (DepthwiseC  (None, 2, 1, 1392)           34800     ['block6e_expand_activation[0]
 onv2D)                                                             [0]']                         
                                                                                                  
 block6e_b

 D)                                                                                               
                                                                                                  
 block7a_expand_bn (BatchNo  (None, 2, 1, 1392)           5568      ['block7a_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block7a_expand_activation   (None, 2, 1, 1392)           0         ['block7a_expand_bn[0][0]']   
 (Activation)                                                                                     
                                                                                                  
 block7a_dwconv (DepthwiseC  (None, 2, 1, 1392)           12528     ['block7a_expand_activation[0]
 onv2D)                                                             [0]']                         
          

 top_activation (Activation  (None, 2, 1, 1536)           0         ['top_bn[0][0]']              
 )                                                                                                
                                                                                                  
 max_pool (GlobalMaxPooling  (None, 1536)                 0         ['top_activation[0][0]']      
 2D)                                                                                              
                                                                                                  
 batch_normalization_24 (Ba  (None, 1536)                 6144      ['max_pool[0][0]']            
 tchNormalization)                                                                                
                                                                                                  
 dense_43 (Dense)            (None, 256)                  393472    ['batch_normalization_24[0][0]
          

In [84]:
history = model.fit(
    data,
    epochs=3,  
    validation_data=(data)  
)

Epoch 1/3
1032/1032 [==============================] - 784s 736ms/step - loss: 5.5149 - accuracy: 0.7772 - val_loss: 2.3879 - val_accuracy: 0.9636
Epoch 2/3
1032/1032 [==============================] - 687s 666ms/step - loss: 1.5591 - accuracy: 0.9358 - val_loss: 0.8070 - val_accuracy: 0.9749
Epoch 3/3
1032/1032 [==============================] - 684s 663ms/step - loss: 0.7469 - accuracy: 0.9558 - val_loss: 0.4963 - val_accuracy: 0.9798


In [86]:
#Train Accuracy
train_loss, train_acc = model.evaluate(data)
print("Train accuracy:",test_acc)

1032/1032 [==============================] - 89s 87ms/step - loss: 0.4963 - accuracy: 0.9798
Train accuracy: 0.9797612428665161


In [103]:
test_data = tf.keras.utils.image_dataset_from_directory(
    url+"\\training_data",
    image_size=(50, 32),
    batch_size=16,
    validation_split=0.1,  
    subset="validation",   
    seed=23
)

Found 20628 files belonging to 36 classes.
Using 2062 files for validation.


In [105]:
test_data = tf.keras.utils.image_dataset_from_directory(
    url+"\\testing_data",
    labels="inferred",
    image_size=(50, 32),
    batch_size=16,
    validation_split=0.1,  # Split the dataset into training and validation
    subset="training",      # Specify "training" or "validation"
    seed=42
)
print(test_data.class_names)

Found 1008 files belonging to 36 classes.
Using 908 files for training.
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [108]:
#Test accuracy
test_loss, test_acc = model.evaluate(test_data)
print("Train accuracy:",test_acc)

57/57 [==============================] - 6s 108ms/step - loss: 0.4557 - accuracy: 0.9934
Train accuracy: 0.9933920502662659


In [ ]:
#Saving the model
model.save("ocnet_model.h5")